<a href="https://colab.research.google.com/github/abou-zithar/ChatBot_Using_Deeplearning/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from copyreg import pickle
import random
import json
import pickle
import numpy as np 
import nltk

'''
# reduce all words to the stem 
work working worked =>work
'''
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [29]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Load Training Data

In [31]:
Lemmatizer=WordNetLemmatizer()

In [32]:
# reading content as text then path it to json.load
intents=json.loads(open('/content/drive/MyDrive/Chatbot/intents.json').read())

In [33]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

In [34]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list =nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


#Prepare Training Data

In [35]:
words=[Lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words=sorted(set(words))

In [36]:
clasees=sorted(set(classes))

In [37]:
pickle.dump(words,open('words.pk1','wb'))
pickle.dump(classes,open('classes.pk1','wb'))

In [38]:
trainning=[]
output_empty=[0]*len(classes)

In [39]:
for doc in documents:
  bag=[]
  word_patterns=doc[0]
  word_patterns=[Lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  trainning.append([bag, output_row])



In [40]:
random.shuffle(trainning)
trainning=np.array(trainning)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [41]:
train_x=list(trainning[:,0])
train_y=list(trainning[:,1])

# Building Model

In [42]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0],),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               31232     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 44)                2860      
                                                                 
Total params: 42,348
Trainable params: 42,348
Non-trainable params: 0
_________________________________________________________________


In [43]:
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [44]:
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('ChatBot_model.hs',hist)
print('done')

Epoch 1/200
26/26 [==============================] - 1s 3ms/step - loss: 3.8079 - accuracy: 0.0312
Epoch 2/200
26/26 [==============================] - 0s 3ms/step - loss: 3.7409 - accuracy: 0.0625
Epoch 3/200
26/26 [==============================] - 0s 3ms/step - loss: 3.6839 - accuracy: 0.0781
Epoch 4/200
26/26 [==============================] - 0s 3ms/step - loss: 3.6074 - accuracy: 0.0703
Epoch 5/200
26/26 [==============================] - 0s 3ms/step - loss: 3.5375 - accuracy: 0.0781
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 3.4600 - accuracy: 0.0781
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 3.3616 - accuracy: 0.1016
Epoch 8/200
26/26 [==============================] - 0s 4ms/step - loss: 3.2656 - accuracy: 0.1406
Epoch 9/200
26/26 [==============================] - 0s 4ms/step - loss: 3.0346 - accuracy: 0.2812
Epoch 10/200
26/26 [==============================] - 0s 7ms/step - loss: 3.0590 - accuracy: 0.2266
Epoch 11/

# building ChatBot

In [45]:
res=model.predict(np.array(train_x))[0]
results=[[i,r] for i,r in enumerate(res) if r> 0.25]
results.sort(key=lambda x: x[1] ,reverse=True)
results_list=[]
for r in results:
  results_list.append({'intent':classes[r[0]],'probability':str(r[1])})

4/4 [==============================] - 0s 3ms/step


In [46]:
def clean_up_sentance(sentance):
  sentance_words=nltk.word_tokenize(sentance)
  wnl = WordNetLemmatizer()
  sentance_words=[wnl.lemmatize(word) for word in sentance_words]
  return sentance_words


In [47]:
def bag_of_words(sentance):
  sentance_words=clean_up_sentance(sentance)
  bag=[0]*len(words)
  for w in sentance_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i]=1
  return np.array(bag)

      

In [48]:
def predict_class(sentance):
  
  bow = np.array(bag_of_words(sentance))
  res=model.predict(bow.reshape(-1,243))[0]
  ERROR_THRESHHOLD=0.25
  results=[[i,r] for i,r in enumerate(res) if r> ERROR_THRESHHOLD]
  results.sort(key=lambda x: x[1] ,reverse=True)

  results_list=[]
  for r in results:
    results_list.append({'intent':classes[r[0]],'probability':str(r[1])})

  return results_list


In [49]:
def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents= intents_json['intents']
  for i in list_of_intents:
    if i['tag'] ==tag:
      result =random.choice(i['responses'])
      break
  return result

In [ ]:
print('___ the BOT is RUNNING......!___')
while True:
  massage=input("")
  ints=predict_class(massage)
  res=get_response(ints,intents)
  print(res)


___ the BOT is RUNNING......!___
hi
1/1 [==============================] - 0s 44ms/step
Hello, thanks for asking
help
1/1 [==============================] - 0s 15ms/step
I can guide you through Adverse management problems, order tracking, person to be contacted and Department related queries
quit
1/1 [==============================] - 0s 15ms/step
...
